# Swissmetro test

In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
import pandas as pd
import pycmtensor as cmt

sm = pd.read_csv("../data/swissmetro.dat", sep="\t")
db = cmt.Data(
    df=sm,
    choice="CHOICE",
    drop=[sm["CHOICE"] == 0],
    autoscale=True,
    autoscale_except=["ID", "ORIGIN", "DEST", "CHOICE"],
    split=0.8,
)

Python 3.9.13 (main, Oct 13 2022, 21:23:06) [MSC v.1916 64 bit (AMD64)] | PyCMTensor 1.3.1


In [2]:
from pycmtensor.expressions import Beta
from pycmtensor.models import MNL

b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
asc_train = Beta("asc_train", 0.0, None, None, 0)
asc_car = Beta("asc_car", 0.0, None, None, 0)
asc_sm = Beta("asc_sm", 0.0, None, None, 1)

U_1 = b_cost * db["TRAIN_CO"] + b_time * db["TRAIN_TT"] + asc_train
U_2 = b_cost * db["SM_CO"] + b_time * db["SM_TT"] + asc_sm
U_3 = b_cost * db["CAR_CO"] + b_time * db["CAR_TT"] + asc_car

# specify the utility function and the availability conditions
U = [U_1, U_2, U_3]  # utility
AV = [db["TRAIN_AV"], db["SM_AV"], db["CAR_AV"]]  # availability

mymodel = MNL(db, locals(), U, AV, name="MNL")
mymodel.config.set_hyperparameter(max_steps=100)

Model training

In [3]:
mymodel.train(db)

11:01:26 [INFO] Start (n=8575)
11:01:35 [INFO] End (t=00:00:10, VE=40.224%, LL=-7529.683422816989, S=23)


Print results and statistics

Beta parameters

In [4]:
print(mymodel.results.beta_statistics())

              value   std err     t-test   p-value rob. std err rob. t-test  \
asc_car   -0.853707  0.047113 -18.120447       0.0     0.001773 -481.494189   
asc_sm          0.0         -          -         -            -           -   
asc_train -1.845829   0.05049 -36.558574       0.0     0.010834 -170.368325   
b_cost     0.025854  0.019678   1.313796  0.188915      0.00523    4.943254   
b_time      -0.5217  0.053767  -9.702955       0.0     0.003695 -141.184022   

          rob. p-value  
asc_car            0.0  
asc_sm               -  
asc_train          0.0  
b_cost        0.000001  
b_time             0.0  


Model statistics

In [5]:
print(mymodel.results.model_statistics())

                                          value
Number of training samples used          8575.0
Number of validation samples used        2143.0
Init. log likelihood               -8885.386433
Final log likelihood               -7529.683423
Accuracy                                 59.78%
Likelihood ratio test                2711.40602
Rho square                             0.152577
Rho square bar                         0.152014
Akaike Information Criterion       15069.366846
Bayesian Information Criterion     15104.649877
Final gradient norm                    0.003011


Correlation matrix

In [6]:
print(mymodel.results.model_correlation_matrix())

             b_cost    b_time  asc_train   asc_car
b_cost     1.000000  0.215873   0.234180 -0.019954
b_time     0.215873  1.000000   0.750515  0.814288
asc_train  0.234180  0.750515   1.000000  0.685223
asc_car   -0.019954  0.814288   0.685223  1.000000


Benchmark

In [7]:
print(mymodel.results.benchmark())

                       value
Seed                     980
Model build time    00:00:13
Model train time    00:00:10
iterations per sec  345.14/s


Predictions

In [8]:
# predictions
print(mymodel.predict(db, return_choices=False))
print(np.unique(mymodel.predict(db), return_counts=True))

[[0.12138601 0.55492529 0.3236887 ]
 [0.10724806 0.5969908  0.29576114]
 [0.10717803 0.60330066 0.28952132]
 ...
 [0.11440961 0.55831256 0.32727783]
 [0.12859771 0.5241943  0.34720798]
 [0.10920566 0.58416641 0.30662793]]
(array([1, 2], dtype=int64), array([2068,   75], dtype=int64))


Elasticities

In [9]:
from pycmtensor.statistics import elasticities

print(elasticities(mymodel, db, 0, "TRAIN_TT"))

[-0.06027963 -0.15127238 -0.20175932 ... -0.02805637 -0.05351249
 -0.04710554]
